In [10]:
from typing import Optional, Tuple
import numpy as np

### Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym** języku programowania.

Pytanko: jakie muszą być wymiary mnożonych macierzy? (Który wymiar musi się "zgadzać"?)

Zadanko: Uzupełnić brakujące wymiary macierzy w docstringu (z dokładnością do "alfa-konwersji")

In [11]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    :param a: matrix with dimensions n by m
    :param b: matrix with dimensions m by p
    :return:  matrix with dimensions n by p
    """
    n = a.shape[0]
    m = a.shape[1]
    p = b.shape[1]
    c = np.zeros((n,p))
    
    for i in range(0,n):
        for j in range(0,p):
            for k in range(0,m):
                c[i,j] += a[i,k] * b[k,j]
    return c
    pass

m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
np.allclose(res, m1 * m2)
#assert np.allclose(res, m1 * m2), "Wrong multiplication result"

True

### Zadania
1. Przeczytać rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
2. Przeczytać rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).
3. Napisać kod (w ulubionym** języku) do eliminacji Gaussa z i bez pivotingu.
4. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki

#### Gauss bez pivotingu

In [12]:
def naive_gauss(a, b):
    """Perform naive Gauss algorithm to solve system of linear equations
    :param a: matrix with dimensions n by n
    :param b: matrix with dimensions 1 by n
    :return:  matrix with dimensions 1 by n
    """
    n = a.shape[0]
    x = np.zeros(n)
    
    for k in range(0,n):
        for i in range(k+1,n):
            xmult = a[i,k] / a[k,k]
            a[i,k] = xmult
            for j in range(k+1,n):
                a[i,j] = a[i,j] - xmult * a[k,j]
            b[i] = b[i] - xmult * b[k]
    x[n-1] = b[n-1] / a[n-1,n-1]
    for i in range(n-1,-1,-1):
        sum = b[i]
        for j in range(i+1,n):
            sum = sum - a[i,j] * x[j]
        x[i] = sum / a[i,i]
    return np.matrix(x).transpose()
    pass

In [18]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()
x1 = np.linalg.solve(A, b)
x = naive_gauss(A, b)
print(x)
np.allclose(x1, x)

[[ 0.21602477]
 [-0.00791511]
 [ 0.63524333]
 [ 0.74617428]]


True

Pytanie: dlaczego wołamy `transpose()` na wektorze `b`? odp.: żeby mieć wektor kolumnowy

Sprawdźmy, czy rozwiązanie jest ok 

(Pytanie': dlaczego po prostu nie użyjemy `==` lub jakiegoś `equals`?) odp.: niedokładność reprezentacji liczb zmiennoprzecinkowych

In [14]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

np.allclose(np.dot(A, x), b)

True

#### Gauss z pivotingiem 

In [15]:
def pivot_gauss(a, b):
    """Perform pivot Gauss algorithm to solve system of linear equations
    :param a: matrix with dimensions n by n
    :param b: matrix with dimensions 1 by n
    :return:  matrix with dimensions 1 by n
    """
    n = a.shape[0]
    x = np.zeros(n)
    s = np.zeros(n)
    d = np.zeros(n, dtype=np.int)
    
    for i in range(0,n):
        d[i] = i
        smax = 0
        for j in range(0,n):
            smax = max(smax, abs(a[i,j]))
        s[i] = smax
        
    for k in range(0,n-1):
        rmax = 0
        for i in range(k,n):
            r = abs(a[d[i],k] / s[d[i]])
            if r > rmax:
                rmax = r
                j = i
        tmp = d[j]
        d[j] = d[k]
        d[k] = tmp
        for i in range(k+1,n):
            xmult = a[d[i],k] / a[d[k],k]
            a[d[i],k] = xmult
            for j in range(k+1,n):
                a[d[i],j] = a[d[i],j] - xmult * a[d[k],j]
    
    for k in range(0,n):
        for i in range(k+1,n):
            b[d[i]] = b[d[i]] - a[d[i],k] * b[d[k]]
    m = n-1
    x[m] = b[d[m]] / a[d[m],m]
    for i in range(n-1,-1,-1):
        sum = b[d[i]]
        for j in range(i+1,n):
            sum = sum - a[d[i],j] * x[j]
        x[i] = sum / a[d[i],i]
    return np.matrix(x).transpose()
    pass

In [20]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()
x1 = np.linalg.solve(A, b)
x = pivot_gauss(A, b)
print(x)
np.allclose(x1, x)

[[ 0.21602477]
 [-0.00791511]
 [ 0.63524333]
 [ 0.74617428]]


True

### Zadania, c.d.
1. Zaimplementować algorytm faktoryzacji LU macierzy
2. (*) Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona
3. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [17]:
def agh_superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    """Perform LU decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    pass

def agh_superfast_check_spd(a: np.matrix) -> bool:
    """Check whether a matrix is symmetric and positive-definite (SPD).
    
    :param a: _
    """
    pass

def agh_superfast_cholesky(a: np.matrix) -> Optional[np.matrix]:
    """Perform a Cholesky decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    pass

### Zadania, opcjonalnie
1. zaimplementować metodę Jacobiego (iteracyjne rozwiązywanie układu równań liniowych)
2. za pomocą tejże metody rozwiązać powyższy układ równań